In [1]:
import pandas as pd
import numpy as np
import time
import itertools
import datetime
import matplotlib.pyplot as plt
import pandas_gbq
from datetime import *
from datetime import datetime, timedelta, date
from pathlib import Path
from zipfile import ZipFile
import urllib
from PIL import Image
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import requests
import json
project_id = "perceptive-ivy-290216"

In [2]:
#Change the date to the previous day
query = """
SELECT
*
FROM
`perceptive-ivy-290216.f1_api.race_lap_time`
# WHERE Year=2024 AND GP="Austrian Grand Prix"
"""
project_id = "perceptive-ivy-290216"
df_bq2 = pandas_gbq.read_gbq(query, project_id=project_id, dialect='standard')

Downloading: 100%|██████████|


In [3]:
df_bq=df_bq2[(df_bq2["GP"]=='Dutch Grand Prix')&(df_bq2["Year"]==2024)]
GP=df_bq
GP["LapNumber"]=GP["LapNumber"].astype(int)
GP=GP.sort_values(['Driver','LapNumber'])
GP["DriverNumber"]=GP["DriverNumber"].astype(int)
year=GP['Year'].iloc[0]
gp=GP['GP'].iloc[0]
GP.head()

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_26930/2706538627.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GP["LapNumber"]=GP["LapNumber"].astype(int)


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,Sector3Time,Sector1SessionTime,Sector2SessionTime,Sector3SessionTime,SpeedI1,SpeedI2,SpeedFL,SpeedST,IsPersonalBest,Compound,TyreLife,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate,Year,GP
5822,0 days 00:59:04.333000,ALB,23,0 days 00:01:28.535000,1,1.0,NaT,NaT,NaT,0 days 00:00:29.206000,0 days 00:00:24.474000,NaT,0 days 00:58:39.991000,0 days 00:59:04.409000,264.0,268.0,301.0,215.0,False,MEDIUM,1.0,True,Williams,0 days 00:57:35.434000,NaT,1,17.0,False,,False,False,2024,Dutch Grand Prix
5321,0 days 01:00:23.447000,ALB,23,0 days 00:01:19.114000,2,1.0,NaT,NaT,0 days 00:00:26.973000,0 days 00:00:28.328000,0 days 00:00:23.813000,0 days 00:59:31.275000,0 days 00:59:59.603000,0 days 01:00:23.416000,282.0,274.0,322.0,284.0,True,MEDIUM,2.0,True,Williams,0 days 00:59:04.333000,NaT,1,17.0,False,,False,True,2024,Dutch Grand Prix
5322,0 days 01:01:41.519000,ALB,23,0 days 00:01:18.072000,3,1.0,NaT,NaT,0 days 00:00:26.760000,0 days 00:00:28.196000,0 days 00:00:23.116000,0 days 01:00:50.176000,0 days 01:01:18.372000,0 days 01:01:41.488000,285.0,273.0,318.0,292.0,True,MEDIUM,3.0,True,Williams,0 days 01:00:23.447000,NaT,1,17.0,False,,False,True,2024,Dutch Grand Prix
5323,0 days 01:02:59.266000,ALB,23,0 days 00:01:17.747000,4,1.0,NaT,NaT,0 days 00:00:26.763000,0 days 00:00:28.171000,0 days 00:00:22.813000,0 days 01:02:08.251000,0 days 01:02:36.422000,0 days 01:02:59.235000,281.0,273.0,317.0,288.0,True,MEDIUM,4.0,True,Williams,0 days 01:01:41.519000,NaT,1,17.0,False,,False,True,2024,Dutch Grand Prix
5324,0 days 01:04:16.704000,ALB,23,0 days 00:01:17.438000,5,1.0,NaT,NaT,0 days 00:00:26.573000,0 days 00:00:28.109000,0 days 00:00:22.756000,0 days 01:03:25.808000,0 days 01:03:53.917000,0 days 01:04:16.673000,277.0,271.0,316.0,305.0,True,MEDIUM,5.0,True,Williams,0 days 01:02:59.266000,NaT,1,17.0,False,,False,True,2024,Dutch Grand Prix


In [4]:
#Convert LapTime to Time Delta for plotting on Y Axis
GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()

# Fastest Driver in each lap

In [5]:
GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()

In [6]:
GP_Rk=GP[GP["LapTime"]!="NaT"]
GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)


In [7]:
GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
GP_Sectors=GP_Sectors.fillna(0)
GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()

,Driver,LapNumber,Time S1 (s),Time S2 (s),Time S3 (s),LapTime,LapTime (s),Compound,Year,GP,LAP_RK,S1_RK,S2_RK,S3_RK
5822,ALB,1,0.0,29.206,24.474,00:01:28.535000,88.535,MEDIUM,2024,Dutch Grand Prix,17.0,0.0,17.0,18.0
5818,ALO,1,0.0,28.360,23.431,00:01:24.726000,84.726,MEDIUM,2024,Dutch Grand Prix,8.0,0.0,10.0,6.0
5827,BOT,1,0.0,28.903,24.316,00:01:27.361000,87.361,SOFT,2024,Dutch Grand Prix,14.0,0.0,13.0,13.0
5817,GAS,1,0.0,28.337,23.663,00:01:24.654000,84.654,MEDIUM,2024,Dutch Grand Prix,7.0,0.0,8.0,9.0
5816,HAM,1,0.0,29.032,24.330,00:01:27.280000,87.280,SOFT,2024,Dutch Grand Prix,13.0,0.0,15.0,14.0


In [8]:
#2024
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="COL":
        return '#64C4FF'    
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [9]:
#get a combined view for fastest drivers in the entire lap, S1, S2, and S3
Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
Fast_Laps=Fast_Laps.reset_index(drop=True)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound
0,Dutch Grand Prix,2024,2,00:01:16.106000,76.106,VER,MEDIUM,26.012,VER,MEDIUM,27.155,NOR,MEDIUM,22.697,PER,MEDIUM
1,Dutch Grand Prix,2024,3,00:01:16.131000,76.131,VER,MEDIUM,25.699,VER,MEDIUM,27.256,MAG,HARD,22.659,HAM,SOFT
2,Dutch Grand Prix,2024,4,00:01:16.131000,76.131,LEC,MEDIUM,25.829,LEC,MEDIUM,27.217,MAG,HARD,22.660,PIA,MEDIUM
3,Dutch Grand Prix,2024,5,00:01:16.181000,76.181,VER,MEDIUM,25.816,NOR,MEDIUM,27.472,GAS,MEDIUM,22.653,PIA,MEDIUM
4,Dutch Grand Prix,2024,6,00:01:16.155000,76.155,VER,MEDIUM,25.711,NOR,MEDIUM,27.425,GAS,MEDIUM,22.475,PER,MEDIUM


In [10]:
Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
Fast_Laps.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
0,Dutch Grand Prix,2024,2,00:01:16.106000,76.106,VER,MEDIUM,26.012,VER,MEDIUM,27.155,NOR,MEDIUM,22.697,PER,MEDIUM,#3860C6
1,Dutch Grand Prix,2024,3,00:01:16.131000,76.131,VER,MEDIUM,25.699,VER,MEDIUM,27.256,MAG,HARD,22.659,HAM,SOFT,#3860C6
2,Dutch Grand Prix,2024,4,00:01:16.131000,76.131,LEC,MEDIUM,25.829,LEC,MEDIUM,27.217,MAG,HARD,22.660,PIA,MEDIUM,#E80020
3,Dutch Grand Prix,2024,5,00:01:16.181000,76.181,VER,MEDIUM,25.816,NOR,MEDIUM,27.472,GAS,MEDIUM,22.653,PIA,MEDIUM,#3860C6
4,Dutch Grand Prix,2024,6,00:01:16.155000,76.155,VER,MEDIUM,25.711,NOR,MEDIUM,27.425,GAS,MEDIUM,22.475,PER,MEDIUM,#3860C6


In [11]:
LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
LapTime_ordered.head()

,GP,Year,LapNumber,LapTime,LapTime (s),Fastest Lap,Lap Compound,Time S1 (s),Fastest S1,S1 Compound,Time S2 (s),Fastest S2,S2 Compound,Time S3 (s),Fastest S3,S3 Compound,color
71,Dutch Grand Prix,2024,72,00:01:13.817000,73.817,NOR,HARD,24.876,NOR,HARD,26.837,NOR,HARD,22.104,NOR,HARD,#FF8000
61,Dutch Grand Prix,2024,62,00:01:13.878000,73.878,HAM,SOFT,24.995,HAM,SOFT,26.733,HAM,SOFT,22.150,HAM,SOFT,#27F4D2
49,Dutch Grand Prix,2024,50,00:01:13.883000,73.883,HAM,SOFT,25.153,NOR,HARD,26.398,HAM,SOFT,22.184,HAM,SOFT,#27F4D2
56,Dutch Grand Prix,2024,57,00:01:13.927000,73.927,RUS,SOFT,25.168,RUS,SOFT,26.545,RUS,SOFT,22.202,HAM,SOFT,#27F4D2
58,Dutch Grand Prix,2024,59,00:01:13.968000,73.968,HAM,SOFT,25.069,RUS,SOFT,26.612,HAM,SOFT,22.080,RUS,SOFT,#27F4D2


In [12]:
import plotly.graph_objects as go
# colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

fig2 = go.Figure(go.Table
 (header={"values": LapTime_ordered.columns},
  cells=dict(
    values=LapTime_ordered.T.values,
    fill_color=[LapTime_ordered.color],
    align='center', font=dict(color='white', size=11)
    ))
 )

# fig2.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": LapTime_ordered.T.values
#                                 if c == "All"
#                                 else LapTime_ordered.loc[LapTime_ordered["LapNumber"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + LapTime_ordered["LapNumber"].unique().tolist()
#             ]
#         }
#     ]
# )

fig2.update_layout(
    title="Fastest Lap and Sector Times Ordered By Lap Times",
        title_x=0.5,
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=13,
        color="Black"
    ),
    height=780,
    width=1200,
    title_font_family="PT Sans Narrow",
    margin=dict(l=5, r=5, t=35, b=5),
)


fig2.show()

In [13]:
fig2.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year,gp),full_html=False, include_plotlyjs='cdn')

In [ ]:
#Multi GP Plot

In [16]:
#2024
def driver_color(driver):
    if driver =="VER":
        return "#3860C6"
    elif driver =="PER":
        return '#3860C6'
    elif driver =="NOR":
        return '#FF8000'
    elif driver =="PIA":
        return '#FF8000'
    elif driver =="LEC":
        return '#E80020'
    elif driver =="SAI":
        return '#E80020'
    elif driver =="HAM":
        return '#27F4D2'
    elif driver =="RUS":
        return '#27F4D2'
    elif driver =="ALO":
        return '#229971'
    elif driver =="STR":
        return '#229971'
    elif driver =="GAS":
        return '#0093cc'
    elif driver =="OCO":
        return '#0093cc'
    elif driver =="MAG":
        return '#B6BABD'
    elif driver =="HUL":
        return '#B6BABD'
    elif driver =="ZHO":
        return '#52e252'
    elif driver =="BOT":
        return '#52e252'
    elif driver =="ALB":
        return '#64C4FF'
    elif driver =="SAR":
        return '#64C4FF'
    elif driver =="COL":
        return '#64C4FF'    
    elif driver =="TSU":
        return '#6692FF'
    elif driver =="RIC":
        return '#6692FF'
    else:
        return '#3860C6'

In [17]:
#2024
race_multi=["Bahrain Grand Prix", "Saudi Arabian Grand Prix", "Australian Grand Prix", "Japanese Grand Prix","Chinese Grand Prix","Miami Grand Prix","Emilia Romagna Grand Prix","Monaco Grand Prix","Canadian Grand Prix","Spanish Grand Prix","Austrian Grand Prix","British Grand Prix","Hungarian Grand Prix","Belgian Grand Prix"]
#2023
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Las Vegas Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Pre-Season Testing","Qatar Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2022
# race_multi=["Abu Dhabi Grand Prix","Australian Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Canadian Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Japanese Grand Prix","Mexico City Grand Prix","Miami Grand Prix","Monaco Grand Prix","Saudi Arabian Grand Prix","Singapore Grand Prix","Spanish Grand Prix","São Paulo Grand Prix","United States Grand Prix"]
#2021
# race_multi=["Abu Dhabi Grand Prix","Austrian Grand Prix","Azerbaijan Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Dutch Grand Prix","Emilia Romagna Grand Prix","French Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Mexico City Grand Prix","Monaco Grand Prix","Portuguese Grand Prix","Qatar Grand Prix","Russian Grand Prix","Saudi Arabian Grand Prix","Spanish Grand Prix","Styrian Grand Prix","São Paulo Grand Prix","Turkish Grand Prix","United States Grand Prix"]
#2020
# race_multi=["70th Anniversary Grand Prix","Abu Dhabi Grand Prix","Austrian Grand Prix","Bahrain Grand Prix","Belgian Grand Prix","British Grand Prix","Eifel Grand Prix","Emilia Romagna Grand Prix","Hungarian Grand Prix","Italian Grand Prix","Portuguese Grand Prix","Russian Grand Prix","Sakhir Grand Prix","Spanish Grand Prix","Styrian Grand Prix","Turkish Grand Prix","Tuscan Grand Prix"]

year_multi=2024
for i in race_multi:

    df_bq=df_bq2[(df_bq2["GP"]==i)&(df_bq2["Year"]==year_multi)]
    GP=df_bq
    GP["LapNumber"]=GP["LapNumber"].astype(int)
    GP=GP.sort_values(['Driver','LapNumber'])
    GP["DriverNumber"]=GP["DriverNumber"].astype(int)
    year=GP['Year'].iloc[0]
    gp=GP['GP'].iloc[0]
    GP['LapTime_TD']= pd.to_timedelta(GP["LapTime"])
    GP["LapTime"]=GP['LapTime'].str.split('days ').str[1]
    GP.loc[:, "LapTime (s)"] = GP["LapTime_TD"].dt.total_seconds()
    GP['s1_TD']= pd.to_timedelta(GP["Sector1Time"])
    GP['s2_TD']= pd.to_timedelta(GP["Sector2Time"])
    GP['s3_TD']= pd.to_timedelta(GP["Sector3Time"])

    GP.loc[:, "Time S1 (s)"] = GP["s1_TD"].dt.total_seconds()
    GP.loc[:, "Time S2 (s)"] = GP["s2_TD"].dt.total_seconds()
    GP.loc[:, "Time S3 (s)"] = GP["s3_TD"].dt.total_seconds()
    GP_Rk=GP[GP["LapTime"]!="NaT"]
    GP_Rk["LAP_RK"] = GP_Rk.groupby("LapNumber")["LapTime (s)"].rank(method="dense", ascending=True)
    GP_Rk["S1_RK"] = GP_Rk.groupby("LapNumber")["Time S1 (s)"].rank(method="dense", ascending=True)
    GP_Rk["S2_RK"] = GP_Rk.groupby("LapNumber")["Time S2 (s)"].rank(method="dense", ascending=True)
    GP_Rk["S3_RK"] = GP_Rk.groupby("LapNumber")["Time S3 (s)"].rank(method="dense", ascending=True)
    GP_Rk["Race_RK"] = GP_Rk.groupby("GP")["LapTime (s)"].rank(method="dense", ascending=True)

    GP_Sectors=GP_Rk[["Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound","Year","GP","LAP_RK","S1_RK","S2_RK","S3_RK"]]
    GP_Sectors=GP_Sectors.fillna(0)
    GP_Sectors.sort_values(by=["LapNumber","S1_RK"]).head()
    Fastest_Lap=GP_Sectors[GP_Sectors["LAP_RK"]==1][["GP","Year","LapNumber","LapTime","LapTime (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_Lap.columns=["GP","Year","LapNumber","LapTime","LapTime (s)","Lap Compound","Fastest Lap"]
    Fastest_sector1=GP_Sectors[GP_Sectors["S1_RK"]==1][["LapNumber","Time S1 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector1.columns=["LapNumber","Time S1 (s)","S1 Compound","Fastest S1"]
    Fastest_sector2=GP_Sectors[GP_Sectors["S2_RK"]==1][["LapNumber","Time S2 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector2.columns=["LapNumber","Time S2 (s)","S2 Compound","Fastest S2"]
    Fastest_sector3=GP_Sectors[GP_Sectors["S3_RK"]==1][["LapNumber","Time S3 (s)","Compound","Driver"]].reset_index(drop=True)
    Fastest_sector3.columns=["LapNumber","Time S3 (s)","S3 Compound","Fastest S3"]
    Fast_Laps=Fastest_Lap.merge(Fastest_sector1,on=['LapNumber']).merge(Fastest_sector2,on=['LapNumber']).merge(Fastest_sector3,on=['LapNumber'])
    Fast_Laps=Fast_Laps[["GP","Year","LapNumber","LapTime","LapTime (s)","Fastest Lap","Lap Compound","Time S1 (s)","Fastest S1","S1 Compound","Time S2 (s)","Fastest S2","S2 Compound","Time S3 (s)","Fastest S3","S3 Compound"]].sort_values(by=["LapNumber"])
    Fast_Laps=Fast_Laps.reset_index(drop=True)
    Fast_Laps.head()
    Fast_Laps['color'] = Fast_Laps['Fastest Lap'].apply(driver_color)
    Fast_Laps.head()
    LapTime_ordered=Fast_Laps.sort_values(by=['LapTime (s)'])
    LapTime_ordered.head()
    import plotly.graph_objects as go

    fig2 = go.Figure(go.Table
    (header={"values": LapTime_ordered.columns},
    cells=dict(
        values=LapTime_ordered.T.values,
        fill_color=[LapTime_ordered.color],
        align='center', font=dict(color='white', size=11)
        ))
    )
    fig2.update_layout(
        title="Fastest Lap and Sector Times Ordered By Lap Times For The {} {}".format(year_multi, i),
            title_x=0.5,
        yaxis = dict(tickfont = dict(size=15)),
        xaxis = dict(tickfont = dict(size=15)),
        font=dict(
            family="PT Sans Narrow",
            size=13,
            color="Black"
        ),
        height=780,
        width=1200,
        title_font_family="PT Sans Narrow",
        margin=dict(l=5, r=5, t=35, b=5),
    )
    fig2.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Times.html".format(year_multi,i),full_html=False, include_plotlyjs='cdn')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6958/3605435136.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6958/3605435136.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_6958/3605435136.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [14]:
# import plotly.graph_objects as go
# # colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

# fig = go.Figure(go.Table
#  (header={"values": Fast_Laps.columns},
#   cells=dict(
#     values=Fast_Laps.T.values,
#     fill_color=[Fast_Laps.color],
#     align='center', font=dict(color='white', size=11)
#     ))
#  )

# # fig.update_layout(
# #     updatemenus=[
# #         {
# #             "buttons": [
# #                 {
# #                     "label": c,
# #                     "method": "update",
# #                     "args": [
# #                         {
# #                             "cells": {
# #                                 "values": Fast_Laps.T.values
# #                                 if c == "All"
# #                                 else Fast_Laps.loc[Fast_Laps["LapNumber"].eq(c)].T.values
# #                             }
# #                         }
# #                     ],
# #                 }
# #                 for c in ["All"] + Fast_Laps["LapNumber"].unique().tolist()
# #             ]
# #         }
# #     ]
# # )
# fig.update_layout(
#     title="Fastest Lap and Sector Times For Ordered By Lap Number",
#         title_x=0.5,
#         # template="plotly_dark",

#     yaxis = dict(tickfont = dict(size=15)),
#     xaxis = dict(tickfont = dict(size=15)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=13,
#         color="Black"
#     ),
#     height=780,
#     width=1200,
#     title_font_family="PT Sans Narrow",
#     margin=dict(l=5, r=5, t=35, b=5),
# )

# fig.show()

In [162]:
# fig.write_html("{} {} Fastest Lap Times and Sectors Ordered by Lap Number.html".format(year,gp))

In [163]:
# #get a shortened view for fastest drivers in the entire lap, S1, S2, and S3

# Fast_Laps_short=Fast_Laps[["GP","Year","LapNumber","Fastest Lap","Fastest S1","Fastest S2","Fastest S3","color"]].sort_values(by=["LapNumber"])
# Fast_Laps_short=Fast_Laps_short.reset_index(drop=True)
# Fast_Laps_short.head()

In [164]:
# import plotly.graph_objects as go
# # colors = n_colors('rgb(255, 200, 200)', 'rgb(200, 0, 0)', 9, colortype='rgb')

# fig = go.Figure(go.Table
#  (header={"values": Fast_Laps_short.columns},
#   cells=dict(
#     values=Fast_Laps_short.T.values,
#     height=50,
#     fill_color=[Fast_Laps_short.color],
#     align='center', font=dict(color='white', size=20)
#     ))
#  )

# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": Fast_Laps_short.T.values
#                                 if c == "All"
#                                 else Fast_Laps_short.loc[Fast_Laps_short["LapNumber"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + Fast_Laps_short["LapNumber"].unique().tolist()
#             ]
#         }
#     ]
# )
# fig.update_layout(
#     title="Fastest Lap and Sector Times",
#         title_x=0.5,
#     yaxis = dict(tickfont = dict(size=20)),
#     xaxis = dict(tickfont = dict(size=20)),
#     font=dict(
#         family="PT Sans Narrow",
#         size=20,
#         color="Black"
#     ),
#     height=2500,
#     title_font_family="PT Sans Narrow"
# )

# fig.show()

In [107]:
# #get a view for fastest lap in the entire race
# Fastest_Lap_Race=GP_Rk[GP_Rk["Race_RK"]==1][["Year","GP","Driver","LapNumber","Time S1 (s)","Time S2 (s)","Time S3 (s)","LapTime","LapTime (s)","Compound"]].reset_index(drop=True)
# Fastest_Lap_Race

In [108]:
# #Multiple plotly filters code
# import plotly.graph_objects as go

# df = pd.DataFrame(
#     {
#         "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
#         "A_item": [2, 8, 0],
#         "B_item": [1, 7, 10],
#         "C_item": [9, 2, 9],
#         "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
#     }
# )

# fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
# fig.update_layout(
#     updatemenus=[
#         {
#             "y": 1 - (i / 5),
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "restyle",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": df.T.values
#                                 if c == "All"
#                                 else df.loc[df[menu].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + df[menu].unique().tolist()
#             ],
#         }
#         for i, menu in enumerate(["Channel_type", "Date"])
#     ]
# )

In [109]:
# #Single Plotly filter code
# import plotly.graph_objects as go

# df = pd.DataFrame(
#     {
#         "Date": ["2020-01-27", "2020-02-27", "2020-03-27"],
#         "A_item": [2, 8, 0],
#         "B_item": [1, 7, 10],
#         "C_item": [9, 2, 9],
#         "Channel_type": ["Channel_1", "Channel_1", "Channel_2"],
#     }
# )

# fig = go.Figure(go.Table(header={"values": df.columns}, cells={"values": df.T.values}))
# fig.update_layout(
#     updatemenus=[
#         {
#             "buttons": [
#                 {
#                     "label": c,
#                     "method": "update",
#                     "args": [
#                         {
#                             "cells": {
#                                 "values": df.T.values
#                                 if c == "All"
#                                 else df.loc[df["Channel_type"].eq(c)].T.values
#                             }
#                         }
#                     ],
#                 }
#                 for c in ["All"] + df["Channel_type"].unique().tolist()
#             ]
#         }
#     ]
# )